In [1]:
import requests
import pandas as pd
from configparser import ConfigParser

In [2]:
username = 'DS3002API'
client_id = 'no9xCWocGbF-E_D-6U8Meg'

In [3]:
parser = ConfigParser()

try:
    _ = parser.read('notebook.cfg')
except:
    print('Unable to read configuration file')
    
try:
    secret_key = parser.get('my_api', 'secret_key')
    pw = parser.get('my_api', 'pw')
except:
    print('Unable to fetch secret key and/or password')

In [4]:
try:
    auth = requests.auth.HTTPBasicAuth(client_id, secret_key)
except:
    print('Unable to authenticate client id and/or secret key')

In [5]:
data = {
    'grant_type': 'password',
    'username': username,
    'password': pw
}

headers = {'User-Agent': 'My DS3002 API'}

In [6]:
try:
    res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)
    token = res.json()['access_token']
except:
    print('Unable to retrieve access token')

In [7]:
headers['Authorization'] = f'bearer {token}'

In [8]:
sub = input('What subreddit would you like to fetch posts from?: ')
limit = int(input('How many posts would you like to retrieve (1-100)?: '))
time = input('I would like to view the top posts from the past (hour, day, week, month, year, all): ').lower()

What subreddit would you like to fetch posts from?:  news
How many posts would you like to retrieve (1-100)?:  50
I would like to view the top posts from the past (hour, day, week, month, year, all):  month


In [9]:
try:
    r = requests.get(f'https://oauth.reddit.com/r/{sub}/top', headers=headers, params={'t': time, 'limit': limit}).json()
except:
    print('Unable to fetch posts')

In [10]:
l = r['data']['children']

In [11]:
df = pd.DataFrame(data={'Title': [], 'Number of Upvotes': [], 'Percent Upvoted': [], 'Number of Comments': [], 
                        'Link': []}).astype({"Number of Upvotes": int, "Number of Comments": int})

In [12]:
for i in range(0, len(l)):
    j = l[i]['data']
    temp_df = pd.DataFrame(data={'Title': j['title'], 'Number of Upvotes': int(j['score']), 
                                 'Percent Upvoted': j['upvote_ratio'], 
                                 'Number of Comments': int(j['num_comments']), 
                                 'Link': j['url_overridden_by_dest']}, index=[0])
    df = df.append(temp_df)
df = df.reset_index(drop=True)

In [13]:
df['Upvote to Comment Ratio'] = round(df['Number of Upvotes'] / df['Number of Comments'], 2)
df['Controversy Rating'] = 1 / (df['Percent Upvoted'] * df['Upvote to Comment Ratio'])
df = df.sort_values('Controversy Rating', ascending=False)

In [14]:
df

,Title,Number of Upvotes,Percent Upvoted,Number of Comments,Link,Upvote to Comment Ratio,Controversy Rating
32,Utah bans transgender athletes in girls sports...,55751,0.83,13803,https://apnews.com/article/3ffc9205bfbeb05ae3a...,4.04,0.298223
20,New Secret Service report details growing ince...,65843,0.80,11642,https://www.cbsnews.com/news/incel-threat-secr...,5.66,0.220848
46,"National average for a gallon of gas tops $4, ...",42920,0.90,6445,https://www.cnbc.com/2022/03/06/national-avera...,6.66,0.166834
47,"U.S. home sales tumble; higher prices, mortgag...",40744,0.94,6127,https://www.reuters.com/world/us/us-existing-h...,6.65,0.159974
5,"As inflation heats up, 64% of Americans are no...",91960,0.90,12945,https://www.cnbc.com/2022/03/08/as-prices-rise...,7.10,0.156495
42,Plane carrying Donald Trump made emergency lan...,48055,0.79,5677,https://www.washingtonpost.com/nation/2022/03/...,8.46,0.149624
40,Foo Fighters' Taylor Hawkins had 10 different ...,51698,0.88,5666,https://www.cbsnews.com/amp/news/taylor-hawkin...,9.12,0.124601
6,Russia's 40-mile convoy has stalled on its way...,90269,0.92,8936,https://www.npr.org/2022/03/01/1083733700/russ...,10.10,0.107619
33,Russian fast-food chain backed by parliament t...,55291,0.92,4790,https://www.independent.co.uk/news/world/europ...,11.54,0.094190
22,Capitol riot suspect is granted refugee status...,65602,0.86,5227,https://www.cnn.com/2022/03/22/politics/evan-n...,12.55,0.092653


In [15]:
export = input("Would you like to export to csv? (y/n): ").lower()

Would you like to export to csv? (y/n):  n


In [16]:
if export:
    try:
        df.to_csv('output.csv', index=False)
    except:
        print('Unable to export')